In [ ]:
from rdkit import Chem
from rdkit.Chem import rdFingerprintGenerator
from rdkit.Chem.Draw import IPythonConsole
from rdkit import DataStructs
import rdkit
import os 

import matplotlib.pyplot as plt

from sklearn.cross_decomposition import PLSRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import permutation_importance
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import (LeaveOneOut, cross_val_predict,
                                     cross_val_score, train_test_split)

import numpy as np 
import re

from rdkit.Chem import Descriptors
#print(len(Descriptors._descList))
#print(Descriptors._descList[:5])
#print(rdkit.__version__)

from contextlib import contextmanager,redirect_stderr,redirect_stdout
from os import devnull

@contextmanager
def suppress_stdout_stderr():
    """A context manager that redirects stdout and stderr to devnull"""
    with open(devnull, 'w') as fnull:
        with redirect_stderr(fnull) as err, redirect_stdout(fnull) as out:
            yield (err, out)

DEfine basic data structures and config

In [ ]:
# those are the output data
molnames = []
labels = []
diffs_toothermethods = []
chemical_reacts = []
stechio_ceofs = []
moldescriptors = []

howmanydifs = 3
rootdirqdata = './qdata/'
rootdirdata = './data/'

hflist = ["Nuclear Repulsion  :", \
          "One Electron Energy:", \
          "Two Electron Energy:", \
          "Potential Energy   :", \
          "Kinetic Energy     :", \
          "Dispersion correction", \
          "Total Charge", \
          "Multiplicity", \
          "Number of Electrons", \
          "FINAL SINGLE POINT ENERGY"]

pbelist = ["Nuclear Repulsion  :", \
            "One Electron Energy:", \
            "Two Electron Energy:", \
            "Potential Energy   :", \
            "Kinetic Energy     :", \
            "E(X)               :"  , \
            "E(C)               :"  , \
            "Dispersion correction", \
            "Total Charge"   , \
            "Multiplicity"   , \
            "Number of Electrons", \
            "FINAL SINGLE POINT ENERGY"]


Read molecules labels and more

In [ ]:

fp = open(rootdirdata + '/labels.txt', 'r')

for line in fp:
    sline = line.replace("\t", " ").replace("\n", "").rstrip().lstrip().split()
    molname = sline[1]
    
    difvals = []   
    for i in range(howmanydifs):
        difvals.append(float(sline[-1*(i+1)]))

    schechio = []
    reacts = []
    for i in range(2,len(sline)-howmanydifs-1):
        nospace = sline[i].replace(" ", "")
        if nospace.isdigit():
            schechio.append(int(nospace))
        elif nospace.startswith("-") and nospace[1:].isdigit():
            schechio.append(int(nospace))
        else:
            reacts.append(nospace)

    stechio_ceofs.append(schechio)
    chemical_reacts.append(reacts)
    diffs_toothermethods.append(difvals)
    labels.append(float(sline[-1*howmanydifs-1]))
    molnames.append(molname)
    moldescriptors.append({})

fp.close()

print("")
print("Number of labels         :", len(labels))
print("Number of names          :", len(molnames))
print("Number of differences    :", len(diffs_toothermethods))
print("Number of chemicals      :", len(chemical_reacts))
print("Number of stechio ceofs  :", len(stechio_ceofs))
print("Number of moldescriptors :", len(moldescriptors))

Read PBE data

In [ ]:
pbedescriptor = {}

for file in os.listdir(rootdirqdata+'/PBE/'):
    if file.endswith('.out'):
        molname = file.split('.out')[0]
        molname = re.split("\.mpi\d+", molname)[0]
        #print(molname)
        moldesc = {}
        fp = open(rootdirqdata+'/PBE/'+file, 'r')
        for line in fp:
            for val in pbelist:
                if line.find(val) != -1:
                    keyval = val.replace(":", "").rstrip().lstrip().replace(" ", "_")
                    sline = line.rstrip().lstrip().split()
                    for sval in sline:
                        try:
                            firstnumvalue = float(sval)
                            break
                        except:
                            continue
                    
                    moldesc["PBE_"+keyval] = firstnumvalue
                    #print(molname, keyval, sval)
        fp.close()
        pbedescriptor[molname] = moldesc

for i, molname in enumerate(molnames):
    if molname in pbedescriptor:
        for k in pbedescriptor[molname].keys():
            moldescriptors[i][k] = pbedescriptor[molname][k]
    else:
        print(molname + " not found in PBE descriptors")

print("")
print("Number of labels         :", len(labels))
print("Number of names          :", len(molnames))
print("Number of differences    :", len(diffs_toothermethods))
print("Number of chemicals      :", len(chemical_reacts))
print("Number of stechio ceofs  :", len(stechio_ceofs))
print("Number of moldescriptors :", len(moldescriptors))

Read HF data

In [ ]:
hfdescriptor = {}

for file in os.listdir(rootdirqdata+'/HF/'):
    if file.endswith('.out'):
        molname = file.split('.out')[0]
        molname = re.split("\.mpi\d+", molname)[0]
        #print(molname)
        moldesc = {}
        fp = open(rootdirqdata+'/HF/'+file, 'r')
        for line in fp:
            for val in hflist:
                if line.find(val) != -1:
                    keyval = val.replace(":", "").rstrip().lstrip().replace(" ", "_")
                    sline = line.rstrip().lstrip().split()
                    for sval in sline:
                        try:
                            firstnumvalue = float(sval)
                            break
                        except:
                            continue
                    
                    moldesc["HF_"+keyval] = firstnumvalue
                    #print(molname, keyval, sval)
        fp.close()
        hfdescriptor[molname] = moldesc

for i, molname in enumerate(molnames):
    if molname in pbedescriptor:
        for k in hfdescriptor[molname].keys():
            moldescriptors[i][k] = hfdescriptor[molname][k]
    else:
        print(molname + " not found in HF descriptors")


print("")
print("Number of labels         :", len(labels))
print("Number of names          :", len(molnames))
print("Number of differences    :", len(diffs_toothermethods))
print("Number of chemicals      :", len(chemical_reacts))
print("Number of stechio ceofs  :", len(stechio_ceofs))
print("Number of moldescriptors :", len(moldescriptors))

Remove molecules with some missing descriptor

In [ ]:
alldims = set([len(val) for val in moldescriptors])
idxtoremovs = []
for i, val in enumerate(moldescriptors):
    if len(val) != max(alldims):
        idxtoremovs.append(i)

for i in sorted(idxtoremovs, reverse=True):
    print("Molname to remove:", molnames[i], "index:", i)
    del moldescriptors[i]
    del labels[i]
    del molnames[i]
    del diffs_toothermethods[i]
    del chemical_reacts[i]
    del stechio_ceofs[i]

print("")
print("Number of labels         :", len(labels))
print("Number of names          :", len(molnames))
print("Number of differences    :", len(diffs_toothermethods))
print("Number of chemicals      :", len(chemical_reacts))
print("Number of stechio ceofs  :", len(stechio_ceofs))
print("Number of moldescriptors :", len(moldescriptors))

Remove molecules with None Label 

In [ ]:
for i, v in enumerate(labels):
    if v is None:
        print("None value found in labels:", i, molnames[i])
        del moldescriptors[i]
        del labels[i]
        del molnames[i]
        del diffs_toothermethods[i]
        del chemical_reacts[i]
        del stechio_ceofs[i]

print("")
print("Number of labels         :", len(labels))
print("Number of names          :", len(molnames))
print("Number of differences    :", len(diffs_toothermethods))
print("Number of chemicals      :", len(chemical_reacts))
print("Number of stechio ceofs  :", len(stechio_ceofs))
print("Number of moldescriptors :", len(moldescriptors))

If a descriptor is nan at least for a molecule remove from all

In [ ]:
import math 

nandescriptors = set()
for index, molname in enumerate(molnames):
    if any(math.isnan(val) for val in moldescriptors[index].values()):
        print("Nan value found in descriptors:", molname)
        for k,v in moldescriptors[index].items():
            if math.isnan(v):
                nandescriptors.add(k)

print("Removing the following Descriptors ", nandescriptors)
print("Removing ", len(nandescriptors), " descriptors")
for i, v in enumerate(moldescriptors):
    for k in nandescriptors:
        del moldescriptors[i][k]


print("")
print("Number of descriptors:", len(moldescriptors[0]))
print("")
print("Number of labels         :", len(labels))
print("Number of names          :", len(molnames))
print("Number of differences    :", len(diffs_toothermethods))
print("Number of chemicals      :", len(chemical_reacts))
print("Number of stechio ceofs  :", len(stechio_ceofs))
print("Number of moldescriptors :", len(moldescriptors))

Read chemicals

In [ ]:
chemicals_descriptors = {}

for v in chemical_reacts:
    for chem in v:

        moldesc = {}
        fp = open(rootdirqdata + '/PBE/'+chem+'.out', 'r')
        for line in fp:
            for val in pbelist:
                if line.find(val) != -1:
                    keyval = val.replace(":", "").rstrip().lstrip().replace(" ", "_")
                    sline = line.rstrip().lstrip().split()
                    for sval in sline:
                        try:
                            firstnumvalue = float(sval)
                            break
                        except:
                            continue
                    
                    moldesc["PBE_"+keyval] = firstnumvalue

        fp.close

        fp = open(rootdirqdata + '/HF/'+chem+'.out', 'r')
        for line in fp:
            for val in hflist:
                if line.find(val) != -1:
                    keyval = val.replace(":", "").rstrip().lstrip().replace(" ", "_")
                    sline = line.rstrip().lstrip().split()
                    for sval in sline:
                        try:
                            firstnumvalue = float(sval)
                            break
                        except:
                            continue
                    
                    moldesc["HF_"+keyval] = firstnumvalue
        fp.close()

        chemicals_descriptors[chem] = moldesc

print("")
print("Number of chemicals descriptors:", len(chemicals_descriptors))

Check error respect to QM methods

In [ ]:

for methodid in range(howmanydifs):
    y_pred = []
    for i, molname in enumerate(molnames):
        y_pred.append(labels[i] + diffs_toothermethods[i][methodid])
    
    print("Method", methodid+1, "R2 score  :", r2_score(labels, y_pred))
    print("Method", methodid+1, "RMSE score:", mean_squared_error(labels, y_pred, squared=False))

y_pred = []
autokcalmol = 627.5096080305927
for mi, molname in enumerate(molnames):
    #print(molname)
    oury = moldescriptors[mi]["PBE_FINAL_SINGLE_POINT_ENERGY"]
    si = 1
    tosub = 0.0
    for ci, chem in enumerate(chemical_reacts[mi]):
        stecchio = stechio_ceofs[mi][si]
        tosub += stecchio*chemicals_descriptors[chem]["PBE_FINAL_SINGLE_POINT_ENERGY"]
        si += 1

    y_pred.append(autokcalmol*(tosub-oury))
    #print(molname, oury, tosub, 627.51* (tosub-oury), labels[mi])
    

print("Our Method R2 score  :", r2_score(labels, y_pred))
print("Our Method RMSE score:", mean_squared_error(labels, y_pred, squared=False))  
     